# Uplift Modeling with EconML using MovieLens 1M
This notebook downloads MovieLens 1M data, simulates treatment and renewal outcomes, introduces missing data, imputes values, and trains S-, T-, and X-Learners using different base learners.

In [ ]:
!pip uninstall  econml scikit-learn pandas numpy

Found existing installation: econml 0.15.1
Uninstalling econml-0.15.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/econml-0.15.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/econml/*
Proceed (Y/n)? y
  Successfully uninstalled econml-0.15.1
Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/scikit_learn-1.5.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.11/dist-packages/sklearn/*
Proceed (Y/n)? y
  Successfully uninstalled scikit-learn-1.5.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/pandas-2.2.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/pandas/*
Proceed (Y/n)? y
  Successfully uninstalled pandas-2.2.2
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /u

In [3]:
!pip  install --no-cache-dir  econml scikit-learn pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 163.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 211.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.3/147.3 kB 186.5 MB/s eta 0:00:00
  Attempting uninstall: slicer
    Found existing installation: slicer 0.0.8
    Uninstalling slicer-0.0.8:
      Successfully uninstalled slicer-0.0.8
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: shap
    Found existi

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from econml.metalearners import SLearner, TLearner, XLearner

In [2]:
# Download and extract MovieLens 1M dataset
!pip install wget
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

--2025-05-17 18:36:45--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  5.88MB/s    in 1.0s    

2025-05-17 18:36:46 (5.88 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   


In [3]:
# Download and extract MovieLens 1M dataset
# The wget and unzip commands appear to be working correctly based on your output.
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip -o ml-1m.zip -d ml-1m

# Add checks to verify if the directory and file exist
import os

# Correct the path to reflect the nested directory structure
if os.path.exists('ml-1m/ml-1m/ratings.dat'):
    print("ml-1m/ml-1m/ratings.dat found. Proceeding to load data.")
else:
    print("Error: ml-1m/ml-1m/ratings.dat not found. Please check the extraction path.")
    # If the file is still not found after correcting the path, there might be
    # a deeper issue with the unzip process or disk.
    # import sys
    # sys.exit(1) # Uncomment to exit the notebook execution if the file is not found

--2025-05-17 18:37:40--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.2’

ml-1m.zip.2         100%[===================>]   5.64M  5.86MB/s    in 1.0s    

2025-05-17 18:37:41 (5.86 MB/s) - ‘ml-1m.zip.2’ saved [5917549/5917549]

Archive:  ml-1m.zip
  inflating: ml-1m/ml-1m/movies.dat  
  inflating: ml-1m/ml-1m/ratings.dat  
  inflating: ml-1m/ml-1m/README      
  inflating: ml-1m/ml-1m/users.dat   
ml-1m/ml-1m/ratings.dat found. Proceeding to load data.


In [4]:
# Load data
ratings = pd.read_csv('ml-1m//ml-1m/ratings.dat', sep='::', engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
users = pd.read_csv('ml-1m/ml-1m/users.dat', sep='::', engine='python',
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'])
# Specify the encoding as 'latin-1' or 'ISO-8859-1' for the movies.dat file
#movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', engine='python',
#                     names=['MovieID', 'Title', 'Genres'], encoding='latin-1')
#df = ratings.merge(users, on='UserID').merge(movies, on='MovieID')

In [8]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings.shape

(1000209, 4)

In [ ]:
ratings.isnull().values.sum()

0

In [ ]:
ratings.isnull().sum()

,0
UserID,0
MovieID,0
Rating,0
Timestamp,0


In [ ]:
ratings.isnull().any()

,0
UserID,False
MovieID,False
Rating,False
Timestamp,False


In [ ]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   UserID      6040 non-null   int64 
 1   Gender      6040 non-null   object
 2   Age         6040 non-null   int64 
 3   Occupation  6040 non-null   int64 
 4   Zip-code    6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [ ]:
users.isnull().values.sum()

0

In [ ]:
users.value_counts()

,,,,,count
UserID,Gender,Age,Occupation,Zip-code,
1,F,1,10,48067,1
4024,M,25,5,45011,1
4033,M,25,6,43551,1
4032,M,25,12,56301,1
4031,M,25,0,42445,1
...,...,...,...,...,...
2012,M,25,4,49456,1
2011,F,35,3,01545,1
2010,M,18,4,81520,1


In [ ]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
ratings.groupby('UserID')

In [ ]:
ratings.groupby('UserID').get_group(1)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [9]:
ratings.groupby('UserID')['MovieID'].count().rename('MovieID_Count').reset_index().sort_values(by = 'MovieID_Count', ascending =False)

,UserID,MovieID_Count
4168,4169,2314
1679,1680,1850
4276,4277,1743
1940,1941,1595
1180,1181,1521
...,...,...
5724,5725,20
3406,3407,20
1663,1664,20
4418,4419,20


In [ ]:
ratings.groupby('UserID')[['MovieID', 'Rating']].count().rename(columns = {'MovieID':'MovieID_Count','Rating':'Rating_Count' }).reset_index().sort_values(by = 'MovieID_Count', ascending =False)

,UserID,MovieID_Count,Rating_Count
4168,4169,2314,2314
1679,1680,1850,1850
4276,4277,1743,1743
1940,1941,1595,1595
1180,1181,1521,1521
...,...,...,...
5724,5725,20,20
3406,3407,20,20
1663,1664,20,20
4418,4419,20,20


In [10]:
ratings.groupby('UserID').agg({'MovieID':'count', 'Rating':'nunique'}).reset_index().rename(columns = {'MovieID':"MovieID_count", 'Rating':'Unique_Rating_count'})

,UserID,MovieID_count,Unique_Rating_count
0,1,53,3
1,2,129,5
2,3,51,5
3,4,21,5
4,5,198,5
...,...,...,...
6035,6036,888,5
6036,6037,202,5
6037,6038,20,5
6038,6039,123,4


In [ ]:
ratings['UserID'].nunique()

6040

In [ ]:
ratings.nunique()

,0
UserID,6040
MovieID,3706
Rating,5
Timestamp,458455


In [ ]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [ ]:
ratings.head()


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
# Specify the encoding as 'latin-1' or 'ISO-8859-1' for the movies.dat file
movies = pd.read_csv('ml-1m/ml-1m/movies.dat', sep='::', engine='python',
                     names=['MovieID', 'Title', 'Genres'], encoding='latin-1')
#df = ratings.merge(users, on='UserID').merge(movies, on='MovieID')

In [6]:
movies.head(1)

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy


In [ ]:
users.head(1)

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067


In [ ]:
ratings.head(1)

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760


In [7]:
df = ratings.merge(users, on = 'UserID').merge(movies, on = 'MovieID')
df.sample(10)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
821400,4939,1968,4,962790532,M,25,7,63105,"Breakfast Club, The (1985)",Comedy|Drama
532779,3290,2568,2,968093210,M,25,14,90292,"Mod Squad, The (1999)",Action|Crime
730613,4371,3618,3,965180198,F,18,4,63109,Small Time Crooks (2000),Comedy
175875,1112,1231,5,974948871,M,18,6,19104,"Right Stuff, The (1983)",Drama
566293,3476,1367,1,968808011,M,50,0,44126,101 Dalmatians (1996),Children's|Comedy
671567,4033,3755,4,965511626,M,25,6,43551,"Perfect Storm, The (2000)",Action|Adventure|Thriller
767388,4572,1381,3,964462057,F,1,10,17036,Grease 2 (1982),Comedy|Musical|Romance
169624,1077,3360,3,974999470,M,25,3,60660,Hoosiers (1986),Drama
958445,5786,260,4,958142628,M,25,6,53211,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
942567,5686,3066,2,958690060,M,56,16,60610,Tora! Tora! Tora! (1970),War


In [ ]:
users['UserID'].nunique()

6040

In [ ]:
ratings['UserID'].nunique()

6040

In [ ]:
df['UserID'].nunique()

6040

In [ ]:
df.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp', 'Gender', 'Age',
       'Occupation', 'Zip-code', 'Title', 'Genres'],
      dtype='object')

In [8]:
df.dtypes

,0
UserID,int64
MovieID,int64
Rating,int64
Timestamp,int64
Gender,object
Age,int64
Occupation,int64
Zip-code,object
Title,object
Genres,object


In [ ]:
df.Genres.nunique()

301

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   UserID      1000209 non-null  int64 
 1   MovieID     1000209 non-null  int64 
 2   Rating      1000209 non-null  int64 
 3   Timestamp   1000209 non-null  int64 
 4   Gender      1000209 non-null  object
 5   Age         1000209 non-null  int64 
 6   Occupation  1000209 non-null  int64 
 7   Zip-code    1000209 non-null  object
 8   Title       1000209 non-null  object
 9   Genres      1000209 non-null  object
dtypes: int64(6), object(4)
memory usage: 76.3+ MB


In [ ]:
df.dtypes

,0
UserID,int64
MovieID,int64
Rating,int64
Timestamp,int64
Gender,object
Age,int64
Occupation,int64
Zip-code,object
Title,object
Genres,object


In [ ]:
df['Genres'] =df['Genres'].astype('category')
df['Gender'] =df['Gender'].astype('category')
df['Rating'] =df['Rating'].astype('category')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype   
---  ------      --------------    -----   
 0   UserID      1000209 non-null  int64   
 1   MovieID     1000209 non-null  int64   
 2   Rating      1000209 non-null  category
 3   Timestamp   1000209 non-null  int64   
 4   Gender      1000209 non-null  category
 5   Age         1000209 non-null  int64   
 6   Occupation  1000209 non-null  int64   
 7   Zip-code    1000209 non-null  object  
 8   Title       1000209 non-null  object  
 9   Genres      1000209 non-null  category
dtypes: category(3), int64(5), object(2)
memory usage: 57.2+ MB


In [ ]:
df.nunique()

,0
UserID,6040
MovieID,3706
Rating,5
Timestamp,458455
Gender,2
Age,7
Occupation,21
Zip-code,3439
Title,3706
Genres,301


In [10]:
df.head()

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [ ]:
df['Age'].value_counts()

,count
Age,
25,395556
35,199003
18,183536
45,83633
50,72490
56,38780
1,27211


In [11]:
df.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
364045,2121,3,3,974646826,M,56,13,02356,Grumpier Old Men (1995),Comedy|Romance
599503,3648,377,3,994800533,M,18,4,85210,Speed (1994),Action|Romance|Thriller
961878,5795,1747,5,958147550,M,25,1,92688,Wag the Dog (1997),Comedy|Drama
890957,5380,3142,3,960392672,M,18,4,01125,U2: Rattle and Hum (1988),Documentary|Musical
515745,3182,2795,5,968774819,M,25,12,12866,Vacation (1983),Comedy


In [12]:
df.shape, len(df)

((1000209, 10), 1000209)

In [13]:
np.random.seed(88)
np.random.uniform(10,20,len(df))

array([16.47551049, 15.07149688, 15.2834138 , ..., 14.7004425 ,
       19.29304266, 18.10024389])

In [21]:
# Feature creation
np.random.seed(42)
df['WatchTime'] = df['Rating'] * np.random.uniform(15, 30, size=len(df)).astype(int)


In [22]:
df.sample(2)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime
954034,5759,1732,5,959559219,F,25,1,08904,"Big Lebowski, The (1998)",Comedy|Crime|Mystery|Thriller,145
184522,1147,1238,3,974873713,M,25,20,98101,Local Hero (1983),Comedy,60


In [23]:
df['Timestamp'].max(), df['Timestamp'].min()

(1046454590, 956703932)

In [23]:
pd.to_datetime(df['Timestamp'], unit='s')

,Timestamp
0,2000-12-31 22:12:40
1,2000-12-31 22:35:09
2,2000-12-31 22:32:48
3,2000-12-31 22:04:35
4,2001-01-06 23:38:11
...,...
1000204,2000-04-26 02:35:41
1000205,2000-04-25 23:21:27
1000206,2000-04-25 23:19:06
1000207,2000-04-26 02:20:48


In [24]:
df['Timestamp_Date'] = pd.to_datetime(df['Timestamp'], unit='s').dt.strftime('%Y-%m-%d')

In [25]:
df.sample(3)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date
734620,4387,3672,2,965433984,F,18,4,63109,Benji (1974),Adventure|Children's,30,2000-08-05
714252,4277,3753,5,978077207,M,35,16,98133,"Patriot, The (2000)",Action|Drama|War,80,2000-12-29
240635,1449,2952,3,975605335,M,35,20,17112,"Hard 8 (a.k.a. Sydney, a.k.a. Hard Eight) (1996)",Crime|Thriller,63,2000-11-30


In [27]:
df['Timestamp_Date'].max(), df['Timestamp_Date'].min()

('2003-02-28', '2000-04-25')

In [26]:
df['TenureMonths'] = (df['Timestamp'] - df['Timestamp'].min()) // (60*60*24*30)

In [27]:
df.sample(3)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths
535803,3308,3247,4,967977490,F,18,20,15701-1348,Sister Act (1992),Comedy|Crime,112,2000-09-03,4
405577,2429,508,4,975258915,M,25,2,22903,Philadelphia (1993),Drama,72,2000-11-26,7
554198,3410,36,4,967471944,M,35,1,20653,Dead Man Walking (1995),Drama,68,2000-08-28,4


In [30]:
df.head(5)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,103.090509,2000-12-31,8
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,87.782144,2000-12-31,8
2,1,914,3,978301968,F,1,10,48067,My Fair Lady (1964),Musical|Romance,77.939727,2000-12-31,8
3,1,3408,4,978300275,F,1,10,48067,Erin Brockovich (2000),Drama,95.919509,2000-12-31,8
4,1,2355,5,978824291,F,1,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,86.701398,2001-01-06,8


In [28]:
df2 = pd.DataFrame()
df2['UserID'] = df['UserID'].drop_duplicates()
df2.head()

,UserID
0,1
53,2
182,3
233,4
254,5


In [29]:
df2['Age'] = np.random.randint(18, 70, df2.shape[0])
df2.head()

,UserID,Age
0,1,59
53,2,28
182,3,40
233,4,53
254,5,46


In [30]:
df.head(2)

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,100,2000-12-31,8
1,1,661,3,978302109,F,1,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,87,2000-12-31,8


In [31]:
df_user = df.merge(df2, on = 'UserID', how = 'left')
df_user.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Age_x,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age_y
223587,1354,1379,4,974774822,F,25,2,94040,Young Guns II (1990),Action|Comedy|Western,76,2000-11-21,6,27
774591,4620,650,4,964123085,F,18,17,94606,Moll Flanders (1996),Drama,72,2000-07-20,2,67
715286,4284,316,4,965277507,M,50,7,40601,Stargate (1994),Action|Adventure|Sci-Fi,92,2000-08-03,3,25
125270,805,916,5,975396314,M,35,12,37211,Roman Holiday (1953),Comedy|Romance,85,2000-11-28,7,18
44977,303,122,5,976507613,M,25,7,20006,Boomerang (1992),Comedy|Romance,120,2000-12-11,7,32


In [32]:
df_user = df_user.drop('Age_x', axis= 1).rename(columns = {'Age_y':'Age'})
df_user.sample(5)

,UserID,MovieID,Rating,Timestamp,Gender,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age
321371,1904,2717,3,974760635,M,1,30115,Ghostbusters II (1989),Comedy|Horror,87,2000-11-20,6,38
794597,4753,588,1,971726447,M,1,03825,Aladdin (1992),Animation|Children's|Comedy|Musical,20,2000-10-16,5,37
329326,1943,1333,3,974690964,M,4,91501,"Birds, The (1963)",Horror,45,2000-11-20,6,36
944862,5701,1089,4,958588401,M,7,55311,Reservoir Dogs (1992),Crime|Thriller,108,2000-05-17,0,50
583256,3562,2457,3,966788009,F,6,32812,Running Scared (1986),Action|Comedy,75,2000-08-20,3,27


In [33]:
df_user.shape

(1000209, 13)

In [34]:
df_tmp = pd.DataFrame({
    'Category': ['A', 'A', 'B', 'B', 'C'],
    'Value': [10, 20, 30, 40, 50],
    'Other': ['foo', 'bar', 'baz', 'qux', 'quux']
})
df_tmp


,Category,Value,Other
0,A,10,foo
1,A,20,bar
2,B,30,baz
3,B,40,qux
4,C,50,quux


In [35]:
df_tmp.groupby('Category').first()

,Value,Other
Category,,
A,10,foo
B,30,baz
C,50,quux


In [54]:
# Feature creation
user_features = df.groupby('UserID').agg({
    'WatchTime': 'sum',
    'MovieID': 'nunique',
    'TenureMonths': 'max',
    'Age': 'first',
    'Occupation': 'first'
}).rename(columns={'WatchTime': 'TotalWatchTime', 'MovieID': 'UniqueMovies'})

In [37]:
user_features.sample(5)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
1533,6625,75,6,45,1
5682,50407,608,34,18,0
423,3630,64,7,18,4
4118,18559,207,10,25,3
486,3291,39,7,56,0


In [38]:
df_user

,UserID,MovieID,Rating,Timestamp,Gender,Occupation,Zip-code,Title,Genres,WatchTime,Timestamp_Date,TenureMonths,Age
0,1,1193,5,978300760,F,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama,100,2000-12-31,8,59
1,1,661,3,978302109,F,10,48067,James and the Giant Peach (1996),Animation|Children's|Musical,87,2000-12-31,8,59
2,1,914,3,978301968,F,10,48067,My Fair Lady (1964),Musical|Romance,75,2000-12-31,8,59
3,1,3408,4,978300275,F,10,48067,Erin Brockovich (2000),Drama,92,2000-12-31,8,59
4,1,2355,5,978824291,F,10,48067,"Bug's Life, A (1998)",Animation|Children's|Comedy,85,2001-01-06,8,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,6,11106,Weekend at Bernie's (1989),Comedy,28,2000-04-26,0,19
1000205,6040,1094,5,956704887,M,6,11106,"Crying Game, The (1992)",Drama|Romance|War,90,2000-04-25,0,19
1000206,6040,562,5,956704746,M,6,11106,Welcome to the Dollhouse (1995),Comedy|Drama,130,2000-04-25,0,19
1000207,6040,1096,4,956715648,M,6,11106,Sophie's Choice (1982),Drama,96,2000-04-26,0,19


In [40]:
user_features.sample(4)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
2409,11263,134,20,35,9
4038,2073,23,3,50,6
2561,6531,77,8,25,15
5168,2005,20,2,35,7


In [59]:
user_features.sample(frac=0.001).index

Index([2493, 5638, 4262, 1639, 880, 4403], dtype='int64', name='UserID')

In [60]:
# Introduce and impute missing data
user_features.loc[user_features.sample(frac=0.1).index, 'TotalWatchTime'] = np.nan
user_features.sample(10)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
5354,3853.0,46,6.0,45,3
5230,10663.0,130,6.0,25,12
3963,39993.0,457,27.0,50,20
1354,50948.0,684,20.0,25,2
674,7340.0,79,7.0,25,17
505,NaN,115,7.0,35,17
1926,52418.0,585,7.0,1,0
5246,10854.0,130,6.0,18,0
3580,7672.5,72,4.0,18,3


In [56]:
user_features['TotalWatchTime'].isnull().sum()

604

In [61]:
user_features.loc[user_features.sample(frac=0.1).index, 'TenureMonths'] = np.nan
user_features.sample(15)

,TotalWatchTime,UniqueMovies,TenureMonths,Age,Occupation
UserID,,,,,
4305,57145.0,686,NaN,45,17
250,1766.0,20,NaN,35,16
563,NaN,346,7.0,25,4
1052,6352.0,67,7.0,35,2
2491,7613.0,114,6.0,35,7
4231,3940.0,50,NaN,50,18
5338,8658.0,104,1.0,25,17
3889,7672.5,133,NaN,45,6
2637,4186.0,56,6.0,35,20


In [62]:
# Introduce and impute missing data
user_features['TotalWatchTime'] =user_features['TotalWatchTime'].fillna(user_features['TotalWatchTime'].median())
user_features['TenureMonths']= user_features['TenureMonths'].fillna(user_features['TenureMonths'].median())

In [65]:
user_features.isnull().sum()

,0
TotalWatchTime,0
UniqueMovies,0
TenureMonths,0
Age,0
Occupation,0


In [ ]:
# Simulate treatment and renewal
user_features['treatment'] = np.random.binomial(1, 0.5, size=len(user_features))
engaged = user_features['TotalWatchTime'] > user_features['TotalWatchTime'].median()
base_rate = 0.2
uplift = 0.15 * ((user_features['treatment'] == 1) & engaged).astype(float)
user_features['renewed'] = np.random.binomial(1, base_rate + uplift)
X = user_features[['TenureMonths', 'TotalWatchTime', 'UniqueMovies']]
T = user_features['treatment'].values
Y = user_features['renewed'].values

In [ ]:
# Split data
X_train, X_test, T_train, T_test, Y_train, Y_test = train_test_split(X, T, Y, test_size=0.2, random_state=42)

In [ ]:
# Hyperparameter tuning
lr_grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid={'C': [0.01, 0.1, 1, 10]}, cv=3)
lr_grid.fit(X_train, Y_train)
best_lr = lr_grid.best_estimator_

rf_random = RandomizedSearchCV(RandomForestRegressor(random_state=42),
    param_distributions={'n_estimators': [100, 200], 'max_depth': [None, 10, 20]},
    n_iter=4, cv=3, random_state=42)
rf_random.fit(X_train, Y_train)
best_rf = rf_random.best_estimator_

gb_grid = GridSearchCV(GradientBoostingRegressor(random_state=42),
    param_grid={'n_estimators': [100, 150], 'learning_rate': [0.05, 0.1]}, cv=3)
gb_grid.fit(X_train, Y_train)
best_gb = gb_grid.best_estimator_

In [ ]:
# Train learners
s_learner = SLearner(learner=best_lr)
t_learner = TLearner(models=best_rf)
x_learner = XLearner(models=best_gb)
s_learner.fit(Y_train, T_train, X=X_train)
t_learner.fit(Y_train, T_train, X=X_train)
x_learner.fit(Y_train, T_train, X=X_train)
s_te = s_learner.effect(X_test)
t_te = t_learner.effect(X_test)
x_te = x_learner.effect(X_test)
pd.DataFrame({'S_Learner': s_te, 'T_Learner': t_te, 'X_Learner': x_te}).head()